In [ ]:
# Imports
import numpy as np
import torch
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from PIL import Image
import os
from google.colab import files
import shutil

# Set up GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load FaceNet model
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Extract image embeddings
def get_embedding(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model(image)
    return embedding

# Main function to compare images
def process_images():
    # Upload dataset and target image
    print("Upload dataset ZIP.")
    uploaded_dataset = files.upload()
    DATASET_ZIP_NAME = list(uploaded_dataset.keys())[0]
    !unzip {DATASET_ZIP_NAME}
    DATASET_PATH = DATASET_ZIP_NAME[:-4]

    print("Upload target image.")
    uploaded_target = files.upload()
    TARGET_IMAGE_PATH = list(uploaded_target.keys())[0]

    OUTPUT_DIR = '/content/similar_images'
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    target_embedding = get_embedding(TARGET_IMAGE_PATH)

    # Compare and save similar images
    for image_file in os.listdir(DATASET_PATH):
        image_path = os.path.join(DATASET_PATH, image_file)
        current_embedding = get_embedding(image_path)
        distance = torch.dist(target_embedding, current_embedding).item()

        if distance < 0.7:
            shutil.copy2(image_path, OUTPUT_DIR)

    # Zip and download similar images
    shutil.make_archive(OUTPUT_DIR, 'zip', OUTPUT_DIR)
    files.download(f"{OUTPUT_DIR}.zip")

# Run the function
process_images()
